In [1]:
import h2o

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,11 days 22 hours 36 mins
H2O_cluster_timezone:,Asia/Tokyo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.2
H2O_cluster_version_age:,"21 days, 20 hours and 14 minutes"
H2O_cluster_name:,H2O_from_python_root_iycqf9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.549 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [3]:
people = h2o.get_frame( "people" )

In [4]:
#Ratio has been adjusted to get the exact numbers. (see below)
train, test = people.split_frame(
    ratios = [0.897],
    destination_frames = [ "people_train", "people_test" ],
    seed = 123
    )

In [5]:
print( "%d/%d" % ( train.nrows, test.nrows ) )

900/100


In [6]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [7]:
y = "income"
ignoreFields = [ y, "id"]
x = [ i for i in train.names if i not in ignoreFields ]

In [8]:
m3 = H2OGradientBoostingEstimator( model_id = "def9folds", nfolds = 9 )
m3.train( x, y, train )

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [9]:
#Calculating the training error.
m3.mae( train = True ) 

1031.4600260416667

In [10]:
#Calculating the cross-validation error.
m3.mae( xval = True ) 

1340.7100320516324

In [11]:
#Calculating the testing error (against unseen data).
perf = m3.model_performance( test )
perf.mae()

1165.4563814360895

In [12]:
#Overfitting the model by increasing the ntrees and max_depth.
m4 = H2OGradientBoostingEstimator( model_id = "overfit9folds",
                                                         ntrees = 1000,
                                                         max_depth = 10,
                                                         nfolds = 9)
m4.train( x, y, train )

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [13]:
print( "Train: %d  - ->  %d" % (m3.mae(train = True), m4.mae(train = True) ))
print( "Valid: %d  - ->  %d" % (m3.mae(xval = True), m4.mae(xval = True) ))
print( " Test: %d  - ->  %d" % (perf.mae(), m4.model_performance(test).mae() ))

SyntaxError: invalid syntax (<ipython-input-13-0172e32ca265>, line 2)

In [ ]:
#Cross-validation is a better estimate of error!